# DMD

Various preprocessing functions for the DMD (dataset).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from typing import Literal

import matplotlib.pyplot as plt
from tqdm import tqdm

from model.common import create_video_from_images
from model.depth import convert_video_to_depth
from model.dmd import (
    ROOT,
    convert_depth_images_to_video,  # noqa F401
    convert_frames_to_video,
    extract_frames,
    get_all_sessions,
    get_clips,
)

plt.rcParams['font.size'] = 16

In [ ]:
sessions = get_all_sessions()
for session in tqdm(sessions):
    for source_type in ['rgb', 'depth']:
        source_video_extension: Literal['mp4', 'avi'] = (
            'avi' if source_type == 'depth' else 'mp4'
        )
        input_video_path = (
            ROOT / session / f'{session}_{source_type}_body.{source_video_extension}'
        )
        annotations_file_path = ROOT / session / f'{session}.json'

        assert input_video_path.exists(), f'File not found: {input_video_path}'
        assert annotations_file_path.exists(), (
            f'File not found: {annotations_file_path}'
        )
        print(f'Input video path: {input_video_path}')
        print(f'Annotations file path: {annotations_file_path}')

        extract_frames(
            input_video_path=input_video_path,
            annotations_file_path=annotations_file_path,
            force_overwrite=True,
            source_type=source_type,
            skip_output_directory_setup=True,
        )

In [ ]:
# Extract frames from source video based on annotations
extract_frames(
    input_video_path=input_video_path,
    annotations_file_path=annotations_file_path,
    force_overwrite=False,
    source_type=source_type,
    skip_output_directory_setup=False,
)

In [ ]:
# RGB frames to video (also resizes to 518x518)
convert_frames_to_video(session, source_type=source_type, preset='slow', crf=10)

In [ ]:
# Video Depth Anything
convert_video_to_depth(ROOT / session, source_type='crop_rgb')

In [ ]:
convert_depth_images_to_video(
    session, 'normal', 1, extension='png', fps=30, directory='source_depth'
)

## Convert Clip Masks to Video

In [ ]:
session = ROOT / 'gA_4_s1_2019-03-13T10;36;15+01;00'
assert session.exists(), f'Path not found: {session}'

sequences = get_clips(session, 'anomal') + get_clips(session, 'normal')
frame_paths = sorted(
    [img for x in sequences for img in (x / 'masks').glob('*.png')],
    key=lambda x: int(x.stem),
)

In [ ]:
create_video_from_images(
    frame_paths,
    session / f'{session.name}_masks.mp4',
    fps=30,
    size=(256, 256),
)